# Using LLMs for Text Classification
Apart from being used as the basis for conversational agents and chat assistants for accomplishing every day tasks, LLMs can also be used for solving many of the traditional NLP tasks that we mentioned in Lesson 1. They thus can serve as an alternative to prior approaches such as supervised text classification models. In this section, we demonstrate example usage of LLMs for two text classification tasks: 1) sentiment classification of product or movie reviews and 2) topic classification of news article sentences. To highlight the important of evaluation, we also measure the performance of selected LLMs for this task on simple datasets.

## 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

## 2. Model setup

In [2]:
# Pick a model
# model_id = "HuggingFaceTB/SmolLM2-135M" # base model
# model_id = "HuggingFaceTB/SmolLM2-135M-Instruct" # fine-tuned assistant model
# model_id = "HuggingFaceTB/SmolLM3-3B-Base" # base model
model_id= "HuggingFaceTB/SmolLM3-3B" # fine-tuned assistant model
# model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Check if model is loaded correctly
print(f"Model loaded! It has {model.num_parameters():,} parameters")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.26s/it]


Model loaded! It has 3,075,098,624 parameters


## 3. Initialise inference pipeline

In [3]:
# Build pipeline
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

Device set to use mps:0


## 4. Load classification dataset

In [4]:
# Load dataset
data_file = "topic_classification_dataset.csv" # news topic classification
# data_file = "sentiment_classification_dataset.csv" # sentiment classification
texts_df = pd.read_csv(data_file)
texts = texts_df['sentence'].tolist()
ground_truth_labels = texts_df['ground_truth_label'].tolist()
ground_truth_labels_unique = list(set(ground_truth_labels))
print(f"Dataset: {len(texts)} sentences")
print(f"Sample text: {texts[0]}")
print(f"Labels: {set(ground_truth_labels)}")

Dataset: 20 sentences
Sample text: The football team secured a last-minute victory in the championship game.
Labels: {'sports', 'politics', 'entertainment', 'finance', 'science'}


## 5. Setup classification code

In [5]:
# Function to do sentiment classification
def classify_sentiment(text):
    """Classify text using the LLM."""
    prompt = f"Consider the following sentence: '{text}'. Is this a Positive, Negative or Neutral statement?" # sentiment classification
    response = llm(prompt, max_new_tokens=50, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
    response_clean = response.replace(prompt, "").lower()
    
    if "positive" in response_clean:
        return "positive"
    elif "negative" in response_clean:
        return "negative"
    else:
        return "neutral"

# Function to do news topic classification
def classify_topic(text):
    """Classify text using the LLM."""
    prompt = f"Consider the following sentence: '{text}'. Is this a Sports, Finance, Politics, Entertainment or Science statement?" # news topic classification
    response = llm(prompt, max_new_tokens=50, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
    response_clean = response.replace(prompt, "").lower()

    for i in range(0, len(ground_truth_labels_unique)):    
        if ground_truth_labels_unique[i] in response_clean:
            return ground_truth_labels_unique[i]
    return 'sports'

## 6. Do classification

In [6]:
# Classify all sentences
predictions = []
for i, text in enumerate(texts):
    print(f"Processed {i+1}/{len(texts)}")

    if 'sentiment' in data_file:
        pred = classify_sentiment(text)
    else:
        pred = classify_topic(text)
    predictions.append(pred)

print("Classifications complete!")

Processed 1/20
Processed 2/20
Processed 3/20
Processed 4/20
Processed 5/20
Processed 6/20
Processed 7/20
Processed 8/20
Processed 9/20
Processed 10/20
Processed 11/20
Processed 12/20
Processed 13/20
Processed 14/20
Processed 15/20
Processed 16/20
Processed 17/20
Processed 18/20
Processed 19/20
Processed 20/20
Classifications complete!


## 7. Display classifications

In [7]:
print("Predictions:", predictions)
print("Ground truth:", ground_truth_labels)

Predictions: ['sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'finance', 'finance', 'sports', 'sports', 'entertainment', 'sports', 'entertainment', 'sports', 'science', 'sports', 'science']
Ground truth: ['sports', 'sports', 'sports', 'sports', 'politics', 'politics', 'politics', 'politics', 'finance', 'finance', 'finance', 'finance', 'entertainment', 'entertainment', 'entertainment', 'entertainment', 'science', 'science', 'science', 'science']


## 8. Calculate evaluation metrics

In [8]:
# Compute evaluation metrics
accuracy = accuracy_score(ground_truth_labels, predictions)
precision, recall, f1, support = precision_recall_fscore_support(ground_truth_labels, predictions, average='weighted', zero_division=0)

print(f"\nEvaluation Results:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

print(f"\nDetailed Classification Report:")
print(classification_report(ground_truth_labels, predictions, zero_division=0))


Evaluation Results:
Accuracy: 0.500
Precision: 0.657
Recall: 0.500
F1-Score: 0.489

Detailed Classification Report:
               precision    recall  f1-score   support

entertainment       1.00      0.50      0.67         4
      finance       1.00      0.50      0.67         4
     politics       0.00      0.00      0.00         4
      science       1.00      0.50      0.67         4
       sports       0.29      1.00      0.44         4

     accuracy                           0.50        20
    macro avg       0.66      0.50      0.49        20
 weighted avg       0.66      0.50      0.49        20

